In [2]:
# Import dependencies
import os
import json

In [3]:
%pip install --quiet "evadb[document,notebook]"
import evadb
cursor = evadb.connect().cursor()

Note: you may need to restart the kernel to use updated packages.


In [10]:
# Set your OpenAI key as an environment variable
import os
#os.environ['OPENAI_API_KEY'] = ''
open_ai_key = os.environ.get("OPENAI_API_KEY", "")


In [11]:
# set up the extract columns UDF available at functions/extract_columns.py
cursor.query("""CREATE FUNCTION IF NOT EXISTS ExtractColumns
            IMPL  '../evadb/functions/extract_columns.py';
                """).execute()

In [22]:
# # delete the table if it already exists
cursor.query("""DROP TABLE IF EXISTS InputUnstructured
                """).execute()

# create the table specifying the type of the prompt column
cursor.query("""CREATE TABLE IF NOT EXISTS InputUnstructured (
             input_rows TEXT)
                """).execute()

table = cursor.query("SELECT * FROM InputUnstructured;").df()
print(table)

Empty DataFrame
Columns: []
Index: []


In [23]:
input_rows_list = ["The touch screen on my tablet stopped working for no reason.",
#                   "Why does my computer take so long to start up? It's been like this for weeks.",
#                   "My phone battery dies too quickly. I just bought it!",
                    "My headphones won't connect to my phone anymore, even though they used to work just fine.",
                   "The software update completely messed up my computer. Now nothing works properly."]

for input_row in input_rows_list:
    cursor.query(f"""INSERT INTO InputUnstructured (input_rows) VALUES ("{input_row}")""").execute()


In [24]:
table = cursor.query("SELECT * FROM InputUnstructured;").df()
print(table)

   _row_id                                         input_rows
0        1  The touch screen on my tablet stopped working ...
1        2  My headphones won't connect to my phone anymor...
2        3  The software update completely messed up my co...


In [25]:
table = cursor.query("SELECT ExtractColumns(input_rows) FROM InputUnstructured;").df()

for _, row in table.iterrows():
    print(row['response'])
#print(table.iloc[1]['response'])

{
  "Issue Category": "hardware",
  "Raw Issue String": "The touch screen on my tablet stopped working for no reason.",
  "Issue Component": "touch screen"
}
{
  "Issue Category": "hardware",
  "Raw Issue String": "My headphones won't connect to my phone anymore, even though they used to work just fine.",
  "Issue Component": "headphones"
}
{
  "Issue Category": "software",
  "Raw Issue String": "The software update completely messed up my computer. Now nothing works properly.",
  "Issue Component": "computer"
}
